In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

c:\Users\park0\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-hTuzgesB-py3.11\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [ ]:
# 1. 추출할 데이터의 구조를 Pydantic 모델로 정의합니다.
class Ingredient(BaseModel):
    name: str = Field(description="재료의 이름")
    quantity: str = Field(description="재료의 양 (예: 1개, 200g, 1큰술)")


class Recipe(BaseModel):
    recipe_name: str = Field(description="레시피의 이름")
    ingredients: List[Ingredient] = Field(description="필요한 재료 목록")
    instructions: List[str] = Field(description="요리 순서 목록")
    serving_for: int = Field(description="몇 인분 기준인지")


# 2. Pydantic 모델을 사용하여 출력 파서를 생성합니다.
parser = JsonOutputParser(pydantic_object=Recipe)

# 3. LLM에게 지시할 프롬프트 템플릿을 만듭니다.
#    format_instructions는 파서에서 자동으로 생성된 JSON 형식 지침입니다.
prompt = PromptTemplate(
    template="""아래 레시피 텍스트를 분석해서 요청한 JSON 형식으로 정보를 추출해줘.

{format_instructions}

--- 레시피 텍스트 ---
{recipe_text}
""",
    input_variables=["recipe_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 4. 분석할 레시피 텍스트
recipe_text = """
매콤달콤한 닭갈비 레시피 (2인분 기준)

닭갈비를 만들기 위해 먼저 닭다리살 500g을 먹기 좋게 자릅니다. 
그리고 고추장 3큰술, 간장 1큰술, 다진 마늘 1큰술, 설탕 1큰술을 섞어 양념장을 만드세요. 
양배추 1/4개와 고구마 1개를 썰어 준비합니다.
준비된 닭고기를 양념장에 30분간 재워둡니다.
팬에 기름을 두르고 재워둔 닭고기와 썰어둔 채소를 넣고 15분간 볶으면 완성!
"""

In [ ]:
# 모델과 체인 생성
model = ChatOpenAI(model="gpt-4o", temperature=0)
chain = prompt | model | parser

# 체인 실행 및 결과 확인
result = chain.invoke({"recipe_text": recipe_text})

# 결과 출력
import json

print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "recipe_name": "매콤달콤한 닭갈비",
  "ingredients": [
    {
      "name": "닭다리살",
      "quantity": "500g"
    },
    {
      "name": "고추장",
      "quantity": "3큰술"
    },
    {
      "name": "간장",
      "quantity": "1큰술"
    },
    {
      "name": "다진 마늘",
      "quantity": "1큰술"
    },
    {
      "name": "설탕",
      "quantity": "1큰술"
    },
    {
      "name": "양배추",
      "quantity": "1/4개"
    },
    {
      "name": "고구마",
      "quantity": "1개"
    }
  ],
  "instructions": [
    "닭다리살 500g을 먹기 좋게 자릅니다.",
    "고추장 3큰술, 간장 1큰술, 다진 마늘 1큰술, 설탕 1큰술을 섞어 양념장을 만듭니다.",
    "양배추 1/4개와 고구마 1개를 썰어 준비합니다.",
    "준비된 닭고기를 양념장에 30분간 재워둡니다.",
    "팬에 기름을 두르고 재워둔 닭고기와 썰어둔 채소를 넣고 15분간 볶습니다."
  ],
  "serving_for": 2
}
